# Unit and Integration Testing Examples

This notebook demonstrates meaningful vs trivial testing patterns and provides practical examples for achieving proper test coverage in .NET applications.

## 🎯 Meaningful vs Trivial Testing Examples

### ❌ **TRIVIAL** - Basic Property Testing

**Why it's trivial**: Testing simple getters/setters doesn't add value and wastes time.

In [ ]:
// DON'T DO THIS - Trivial property test
public class User
{
    public string Name { get; set; }
    public string Email { get; set; }
}

[Fact]
public void User_SetName_ReturnsName()
{
    // Arrange
    var user = new User();
    var expectedName = "John Doe";
    
    // Act
    user.Name = expectedName;
    
    // Assert
    Assert.Equal(expectedName, user.Name);
}

### ✅ **MEANINGFUL** - Business Logic Testing

**Why it's meaningful**: Tests complex business rules, validation logic, and error conditions.


In [ ]:
// DO THIS - Meaningful business logic test
public class OrderService
{
    private readonly IPaymentProcessor _paymentProcessor;
    private readonly IInventoryService _inventoryService;
    
    public OrderService(IPaymentProcessor paymentProcessor, IInventoryService inventoryService)
    {
        _paymentProcessor = paymentProcessor;
        _inventoryService = inventoryService;
    }
    
    public async Task<OrderResult> ProcessOrderAsync(Order order)
    {
        // Validate order
        if (order.Items.Count == 0)
            throw new InvalidOrderException("Order must contain at least one item");
            
        // Check inventory
        var availabilityResult = await _inventoryService.CheckAvailabilityAsync(order.Items);
        if (!availabilityResult.AllItemsAvailable)
            return OrderResult.Failed("Some items are out of stock");
            
        // Process payment
        var paymentResult = await _paymentProcessor.ProcessPaymentAsync(order.PaymentInfo);
        if (!paymentResult.IsSuccessful)
            return OrderResult.Failed("Payment processing failed");
            
        // Reserve inventory
        await _inventoryService.ReserveItemsAsync(order.Items);
        
        return OrderResult.Success(order.Id);
    }
}

// Meaningful unit tests
public class OrderServiceTests
{
    private readonly Mock<IPaymentProcessor> _mockPaymentProcessor;
    private readonly Mock<IInventoryService> _mockInventoryService;
    private readonly OrderService _orderService;
    
    public OrderServiceTests()
    {
        _mockPaymentProcessor = new Mock<IPaymentProcessor>();
        _mockInventoryService = new Mock<IInventoryService>();
        _orderService = new OrderService(_mockPaymentProcessor.Object, _mockInventoryService.Object);
    }
    
    [Fact]
    public async Task ProcessOrderAsync_EmptyOrder_ThrowsInvalidOrderException()
    {
        // Arrange
        var order = new Order { Items = new List<OrderItem>() };
        
        // Act & Assert
        await Assert.ThrowsAsync<InvalidOrderException>(
            () => _orderService.ProcessOrderAsync(order));
    }
    
    [Fact]
    public async Task ProcessOrderAsync_OutOfStockItems_ReturnsFailedResult()
    {
        // Arrange
        var order = OrderTestData.ValidOrder();
        _mockInventoryService
            .Setup(x => x.CheckAvailabilityAsync(It.IsAny<List<OrderItem>>()))
            .ReturnsAsync(new AvailabilityResult { AllItemsAvailable = false });
            
        // Act
        var result = await _orderService.ProcessOrderAsync(order);
        
        // Assert
        Assert.False(result.IsSuccessful);
        Assert.Contains("out of stock", result.ErrorMessage);
        _mockPaymentProcessor.Verify(x => x.ProcessPaymentAsync(It.IsAny<PaymentInfo>()), Times.Never);
    }
    
    [Fact]
    public async Task ProcessOrderAsync_ValidOrder_ReservesInventoryAndProcessesPayment()
    {
        // Arrange
        var order = OrderTestData.ValidOrder();
        _mockInventoryService
            .Setup(x => x.CheckAvailabilityAsync(It.IsAny<List<OrderItem>>()))
            .ReturnsAsync(new AvailabilityResult { AllItemsAvailable = true });
        _mockPaymentProcessor
            .Setup(x => x.ProcessPaymentAsync(It.IsAny<PaymentInfo>()))
            .ReturnsAsync(new PaymentResult { IsSuccessful = true });
            
        // Act
        var result = await _orderService.ProcessOrderAsync(order);
        
        // Assert
        Assert.True(result.IsSuccessful);
        _mockInventoryService.Verify(x => x.ReserveItemsAsync(order.Items), Times.Once);
        _mockPaymentProcessor.Verify(x => x.ProcessPaymentAsync(order.PaymentInfo), Times.Once);
    }
}

## 🧪 Unit Testing Patterns

### Test Data Builders Pattern

Helps building example object hierarchy for testing

In [ ]:
public class OrderTestData
{
    public static Order ValidOrder() => new OrderBuilder().Build();
    
    public static Order EmptyOrder() => new OrderBuilder()
        .WithItems(new List<OrderItem>())
        .Build();
        
    public static Order LargeOrder() => new OrderBuilder()
        .WithItems(GenerateManyItems(100))
        .Build();
}

public class OrderBuilder
{
    private Order _order;
    
    public OrderBuilder()
    {
        _order = new Order
        {
            Id = Guid.NewGuid(),
            CustomerId = Guid.NewGuid(),
            Items = new List<OrderItem>
            {
                new OrderItem { ProductId = 1, Quantity = 2, Price = 29.99m },
                new OrderItem { ProductId = 2, Quantity = 1, Price = 15.50m }
            },
            PaymentInfo = new PaymentInfo
            {
                CardNumber = "4111111111111111",
                ExpiryDate = DateTime.Now.AddYears(2)
            }
        };
    }
    
    public OrderBuilder WithCustomerId(Guid customerId)
    {
        _order.CustomerId = customerId;
        return this;
    }
    
    public OrderBuilder WithItems(List<OrderItem> items)
    {
        _order.Items = items;
        return this;
    }
    
    public OrderBuilder WithPaymentInfo(PaymentInfo paymentInfo)
    {
        _order.PaymentInfo = paymentInfo;
        return this;
    }
    
    public Order Build() => _order;
}

### Testing Exception Scenarios

In [ ]:
public class UserValidationServiceTests
{
    private readonly UserValidationService _validationService;
    
    public UserValidationServiceTests()
    {
        _validationService = new UserValidationService();
    }
    
    [Theory]
    [InlineData(null)]
    [InlineData("")]
    [InlineData("   ")]
    public void ValidateEmail_InvalidEmail_ThrowsValidationException(string email)
    {
        // Arrange
        var user = new User { Email = email };
        
        // Act & Assert
        var exception = Assert.Throws<UserValidationException>(
            () => _validationService.ValidateEmail(user));
        Assert.Equal("Email is required", exception.Message);
        Assert.Equal("Email", exception.FieldName);
    }
    
    [Theory]
    [InlineData("invalid-email")]
    [InlineData("@domain.com")]
    [InlineData("user@")]
    public void ValidateEmail_MalformedEmail_ThrowsValidationException(string email)
    {
        // Arrange
        var user = new User { Email = email };
        
        // Act & Assert
        var exception = Assert.Throws<UserValidationException>(
            () => _validationService.ValidateEmail(user));
        Assert.Equal("Email format is invalid", exception.Message);
    }
}

## 🔗 Integration Testing Patterns

### Database Integration Tests

In [ ]:
public class UserRepositoryIntegrationTests : IClassFixture<DatabaseFixture>
{
    private readonly DatabaseFixture _fixture;
    private readonly UserRepository _repository;
    
    public UserRepositoryIntegrationTests(DatabaseFixture fixture)
    {
        _fixture = fixture;
        _repository = new UserRepository(_fixture.Context);
    }
    
    [Fact]
    public async Task CreateUserAsync_ValidUser_SavesToDatabase()
    {
        // Arrange
        var user = new User
        {
            Email = "test@example.com",
            Name = "Test User",
            CreatedAt = DateTime.UtcNow
        };
        
        // Act
        var createdUser = await _repository.CreateUserAsync(user);
        
        // Assert
        Assert.NotEqual(Guid.Empty, createdUser.Id);
        
        // Verify it was actually saved
        var savedUser = await _repository.GetUserByIdAsync(createdUser.Id);
        Assert.NotNull(savedUser);
        Assert.Equal(user.Email, savedUser.Email);
        Assert.Equal(user.Name, savedUser.Name);
    }
    
    [Fact]
    public async Task GetUsersByEmailDomainAsync_ExistingDomain_ReturnsMatchingUsers()
    {
        // Arrange
        var domain = "testcompany.com";
        var users = new[]
        {
            new User { Email = "user1@testcompany.com", Name = "User 1" },
            new User { Email = "user2@testcompany.com", Name = "User 2" },
            new User { Email = "user3@otherdomain.com", Name = "User 3" }
        };
        
        foreach (var user in users)
        {
            await _repository.CreateUserAsync(user);
        }
        
        // Act
        var result = await _repository.GetUsersByEmailDomainAsync(domain);
        
        // Assert
        Assert.Equal(2, result.Count());
        Assert.All(result, user => Assert.Contains(domain, user.Email));
    }
}

public class DatabaseFixture : IDisposable
{
    public ApplicationDbContext Context { get; private set; }
    
    public DatabaseFixture()
    {
        var options = new DbContextOptionsBuilder<ApplicationDbContext>()
            .UseInMemoryDatabase(databaseName: Guid.NewGuid().ToString())
            .Options;
            
        Context = new ApplicationDbContext(options);
        Context.Database.EnsureCreated();
    }
    
    public void Dispose()
    {
        Context.Dispose();
    }
}

### API Integration Tests

In [ ]:
public class UsersControllerIntegrationTests : IClassFixture<WebApplicationFactory<Program>>
{
    private readonly WebApplicationFactory<Program> _factory;
    private readonly HttpClient _client;
    
    public UsersControllerIntegrationTests(WebApplicationFactory<Program> factory)
    {
        _factory = factory;
        _client = factory.CreateClient();
    }
    
    [Fact]
    public async Task GetUser_ExistingUser_ReturnsUserData()
    {
        // Arrange
        var userId = await CreateTestUserAsync();
        
        // Act
        var response = await _client.GetAsync($"/api/users/{userId}");
        
        // Assert
        response.EnsureSuccessStatusCode();
        var content = await response.Content.ReadAsStringAsync();
        var user = JsonSerializer.Deserialize<User>(content, new JsonSerializerOptions
        {
            PropertyNameCaseInsensitive = true
        });
        
        Assert.NotNull(user);
        Assert.Equal(userId, user.Id);
    }
    
    [Fact]
    public async Task CreateUser_ValidData_ReturnsCreatedUser()
    {
        // Arrange
        var newUser = new CreateUserRequest
        {
            Email = "newuser@example.com",
            Name = "New User"
        };
        var json = JsonSerializer.Serialize(newUser);
        var content = new StringContent(json, Encoding.UTF8, "application/json");
        
        // Act
        var response = await _client.PostAsync("/api/users", content);
        
        // Assert
        Assert.Equal(HttpStatusCode.Created, response.StatusCode);
        var responseContent = await response.Content.ReadAsStringAsync();
        var createdUser = JsonSerializer.Deserialize<User>(responseContent, new JsonSerializerOptions
        {
            PropertyNameCaseInsensitive = true
        });
        
        Assert.NotNull(createdUser);
        Assert.NotEqual(Guid.Empty, createdUser.Id);
        Assert.Equal(newUser.Email, createdUser.Email);
        Assert.Equal(newUser.Name, createdUser.Name);
    }
    
    [Fact]
    public async Task CreateUser_InvalidEmail_ReturnsBadRequest()
    {
        // Arrange
        var invalidUser = new CreateUserRequest
        {
            Email = "invalid-email",
            Name = "Test User"
        };
        var json = JsonSerializer.Serialize(invalidUser);
        var content = new StringContent(json, Encoding.UTF8, "application/json");
        
        // Act
        var response = await _client.PostAsync("/api/users", content);
        
        // Assert
        Assert.Equal(HttpStatusCode.BadRequest, response.StatusCode);
    }
    
    private async Task<Guid> CreateTestUserAsync()
    {
        var user = new CreateUserRequest
        {
            Email = "test@example.com",
            Name = "Test User"
        };
        var json = JsonSerializer.Serialize(user);
        var content = new StringContent(json, Encoding.UTF8, "application/json");
        
        var response = await _client.PostAsync("/api/users", content);
        var responseContent = await response.Content.ReadAsStringAsync();
        var createdUser = JsonSerializer.Deserialize<User>(responseContent, new JsonSerializerOptions
        {
            PropertyNameCaseInsensitive = true
        });
        
        return createdUser.Id;
    }
}

## 📊 Code Coverage Strategies

### Setting Up Coverage with Coverlet

```xml
<!-- In test project .csproj -->
<PackageReference Include="coverlet.collector" Version="3.1.2">
  <PrivateAssets>all</PrivateAssets>
  <IncludeAssets>runtime; build; native; contentfiles; analyzers</IncludeAssets>
</PackageReference>
<PackageReference Include="coverlet.msbuild" Version="3.1.2">
  <PrivateAssets>all</PrivateAssets>
  <IncludeAssets>runtime; build; native; contentfiles; analyzers</IncludeAssets>
</PackageReference>
```

### Running Tests with Coverage

In [ ]:
# Command line coverage
dotnet test --collect:"XPlat Code Coverage"

# Generate HTML report
dotnet test --collect:"XPlat Code Coverage" --results-directory:./TestResults
reportgenerator -reports:"./TestResults/**/coverage.cobertura.xml" -targetdir:"./TestResults/html" -reporttypes:Html

# Check coverage threshold
dotnet test /p:CollectCoverage=true /p:CoverletOutputFormat=cobertura /p:Threshold=50

### Coverage Configuration

```xml
<!-- coverlet.runsettings -->
<?xml version="1.0" encoding="utf-8" ?>
<RunSettings>
  <DataCollectionRunSettings>
    <DataCollectors>
      <DataCollector friendlyName="XPlat code coverage">
        <Configuration>
          <Format>cobertura</Format>
          <Exclude>[*.Tests]*,[*]*.Migrations.*</Exclude>
          <ExcludeByAttribute>Obsolete,GeneratedCode,CompilerGenerated</ExcludeByAttribute>
          <ExcludeByFile>**/Program.cs</ExcludeByFile>
          <IncludeDirectory>../src</IncludeDirectory>
        </Configuration>
      </DataCollector>
    </DataCollectors>
  </DataCollectionRunSettings>
</RunSettings>
```

## 🚀 Advanced Testing Patterns

### Testing Async Operations

In [ ]:
public class AsyncServiceTests
{
    [Fact]
    public async Task ProcessDataAsync_LargeDataset_CompletesWithinTimeout()
    {
        // Arrange
        var service = new DataProcessingService();
        var largeDataset = GenerateLargeDataset(10000);
        var timeout = TimeSpan.FromSeconds(5);
        
        // Act & Assert
        using var cts = new CancellationTokenSource(timeout);
        var result = await service.ProcessDataAsync(largeDataset, cts.Token);
        
        Assert.NotNull(result);
        Assert.True(result.ProcessedCount > 0);
    }
    
    [Fact]
    public async Task ProcessDataAsync_CancellationRequested_ThrowsOperationCanceledException()
    {
        // Arrange
        var service = new DataProcessingService();
        var dataset = GenerateLargeDataset(1000000); // Very large dataset
        using var cts = new CancellationTokenSource();
        
        // Act
        var task = service.ProcessDataAsync(dataset, cts.Token);
        cts.Cancel(); // Cancel immediately
        
        // Assert
        await Assert.ThrowsAsync<OperationCanceledException>(() => task);
    }
}

## 🏗️ Test Organization and Structure

### Project Structure Example

```
MyApp.Solution/
├── src/
│   ├── MyApp.Core/           # Domain models and interfaces
│   ├── MyApp.Infrastructure/ # Data access, external services
│   ├── MyApp.Api/           # Web API controllers
│   └── MyApp.Web/           # Frontend application
└── tests/
    ├── MyApp.Core.Tests/           # Unit tests for core logic
    ├── MyApp.Infrastructure.Tests/ # Integration tests for data access
    ├── MyApp.Api.Tests/           # API integration tests
    └── MyApp.Integration.Tests/    # End-to-end integration tests
```